### RQ2（仮）誰がSATDを除去するのか？

In [1]:
from setting import PATH_OF_UNIQUE_SATD_INFO, PATH_OF_ALLPROJECT_CSV, PATH_OF_GITCLONE
from tqdm import tqdm
import os
import re
import pandas as pd
from datetime import datetime
import subprocess

In [2]:
df = pd.read_csv('./a-extractComment/SATDDeletedWithInfo.csv', index_col=0)
df.head()

,project,gitPath,CommitID,Deleted CommitID,Author,Delete Author,LatestDockerfile,Comment,CommitDate,DeleteComment,(File) FirstCommit,(File) Deleted,firstCommitからコメント追加までの日数,コメント追加からコメント削除までの日数,追加時ファイル,削除時ファイル,(origin)追加時ファイル,(origin)SATD
313,wikimedia\nmediawiki-docker,https://github.com/wikimedia/mediawiki-docker,4b8cc73503e4fadd6f2b01eba840fa787a0d0ee3,b48d43686790d3497fec7a7607d640f9b8c43d67,Gabriel Wicke <gwicke@wikimedia.org>,David Barratt <dbarratt@wikimedia.org>,1.31/apache/Dockerfile,# Extensions\n # TODO: make submodules ...,2015/12/19 10:41,2017/08/05 02:00,2015/08/13 14:16,NaN,"127 days, 20:25:00","594 days, 15:19:00",https://github.com/wikimedia/mediawiki-docker/...,https://github.com/wikimedia/mediawiki-docker/...,https://github.com/wikimedia/mediawiki-docker/...,# VisualEditor\n # TODO: make submodule...
178,31z4\nzookeeper-docker,https://github.com/31z4/zookeeper-docker,15b6f73b9f36a8bf4960c661427ac41de49cf534,2ac61a0d18eb23c113c38a57380548a000768cb7,Duncan Schulze <duncan.schulze@sonian.net>,Duncan Schulze <dschulze@barracuda.com>,3.5.8/Dockerfile,# TODO: the distribution given by apache i...,2017/10/24 14:12,2018/06/01 00:36,2017/10/24 14:12,NaN,0 days 00:00:00.000000000,"219 days, 10:24:00",https://github.com/31z4/zookeeper-docker/commi...,https://github.com/31z4/zookeeper-docker/commi...,NaN,NaN
255,rocker-org\nrocker,https://github.com/rocker-org/rocker,a35c1df7e3dced0956cc4c33fd32ef71a443d8c4,1617fbefba8f6e7cdbbfcdfb6f77c5653249ecb2,Carl Boettiger <cboettig@gmail.com>,Carl Boettiger <cboettig@gmail.com>,rstudio/testing/Dockerfile,## hack to avoid RStudio-bug in detecting bu...,2016/12/22 03:57,2016/12/23 00:55,2016/11/12 15:45,NaN,39 days 12:12:00.000000000,20:58:00,https://github.com/rocker-org/rocker/commit/a3...,https://github.com/rocker-org/rocker/commit/16...,NaN,NaN
121,docker\ndocker-ce,https://github.com/docker/docker-ce,2a1e2c7de1d97b0f8123e0e7d0cd79d4efa10c80,7fe655caac4b56da0004a5a7c52765b7e09aef1f,Tibor Vass <tibor@docker.com>,Antonio Murdaca <runcom@redhat.com>,components/engine/Dockerfile,# !!! TEMPORARY HACK !!!\n# Because of https:/...,2016/04/16 10:00,2016/05/25 18:58,2013/07/09 13:30,NaN,1011 days 20:30:00.000000000,"39 days, 8:58:00",https://github.com/docker/docker-ce/commit/2a1...,https://github.com/docker/docker-ce/commit/7fe...,NaN,NaN
139,docker\ndocker-ce,https://github.com/docker/docker-ce,7a1680b13a4d40f402fb54bfef604e8539c37662,dd52c68ae4d8dbcbb34a4ba7983eb160dbfe37fc,Brett Randall <javabrett@gmail.com>,Brian Goff <cpuguy83@gmail.com>,components/engine/Dockerfile,# # Make a dockerd binary.\n# # hack/make.sh b...,2018/02/14 20:08,2019/10/04 06:07,2013/07/09 13:30,NaN,"1681 days, 6:38:00","596 days, 9:59:00",https://github.com/docker/docker-ce/commit/7a1...,https://github.com/docker/docker-ce/commit/dd5...,https://github.com/docker/docker-ce/commit/1d5...,# # Make a dockerd binary.\n# # hack/make.sh b...


In [3]:
def daterange_to_minute(d):
    if "days," in d:
        days_time = d.split(" days, ")
        days = days_time[0]
        time = days_time[1].split(":")
        return int(days)*24*60 + int(time[0])*60 + int(time[1])
    elif "day," in d:
        days_time = d.split(" day, ")
        days = days_time[0]
        time = days_time[1].split(":")
        return int(days)*24*60 + int(time[0])*60 + int(time[1])
    elif "days" in d:
        days_time = d.split(" days ")
        days = days_time[0]
        time = days_time[1].split(":")
        return int(days)*24*60 + int(time[0])*60 + int(time[1])
    else:
        time = d.split(":")
        return int(time[0])*60 + int(time[1])
    
    
df["コメント追加からコメント削除までの日数"] = df["コメント追加からコメント削除までの日数"].apply(lambda dt: daterange_to_minute(dt))

In [4]:
# コミット者を保持する配列
commiter = { "add_satd":[], "delete_satd":[] }

unique_satds = df["Comment"].unique()

for satd in unique_satds:
    tmp = df[df["Comment"] == satd]
    # 追加までの日数で絞る
    tmp = tmp.sort_values("コメント追加からコメント削除までの日数")
    commiter["add_satd"].append(tmp.iloc[-1, :]["Author"])
    commiter["delete_satd"].append(tmp.iloc[-1, :]["Delete Author"])

In [5]:
commiter

{'add_satd': ['Gabriel Wicke <gwicke@wikimedia.org>',
  'Duncan Schulze <duncan.schulze@sonian.net>',
  'Carl Boettiger <cboettig@gmail.com>',
  'Tibor Vass <tibor@docker.com>',
  'Brett Randall <javabrett@gmail.com>',
  'Gerhard Lazu <gerhard@lazu.co.uk>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Gerhard Lazu <gerhard@lazu.co.uk>',
  'Mikael Arguedas <mikael.arguedas@gmail.com>',
  'Vincent Demeester <vincent@sbr.pm>',
  'ruffsl <roxfoxpox@gmail.com>',
  'Alexander Morozov <lk4d4@docker.com>',
  'Brian Goff <cpuguy83@gmail.com>',
  'Kir Kolyshkin <kolyshkin@gmail.com>',
  'Gerhard Lazu <gerhard@lazu.co.uk>',
  'Gerhard Lazu <gerhard@lazu.co.uk>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwiggin@gmail.com>',
  'Tianon Gravi <admwi

In [6]:
count = 0

for i in range(len(unique_satds)):
    if commiter["add_satd"][i] == commiter["delete_satd"][i]:
        count += 1

print(f"コミット者が同一であった件数: {count}/{len(unique_satds)}")

コミット者が同一であった件数: 35/57
